In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
%matplotlib inline
import sys
sys.path.append("/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Code")
np.set_printoptions(precision=5, suppress=True)

In [ ]:
ds_path = '/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Survey+dataset/3_filtered_values.csv'

df = pd.read_csv(ds_path)

In [ ]:
df

,Country,Gender,Education,Profession,Work_status,Household_members,Income_level,Location_of_resudence,Centre_or_suburbs,Public_transport_service,Car_driving_license,Considering_electric_or_hybrid_vehicle_next_purchase,Know_what_car_sharing_is,Would_subsribe_car_sharing_if_available,Most_frequent_trip_Walk,Most_frequent_trip_Bicycle,Most_frequent_trip_Car_as_Driver,Most_frequent_trip_Car_as_Passenger,Most_frequent_trip_Train,Most_frequent_trip_Underground_or_light_train,Most_frequent_trip_Tram,Most_frequent_trip_Bus,Most_frequent_trip_Motorcycle_or_moped,Destination_most_frequent_trip,Frequency_most_frequent_trip,Frequent_trip_distance,Concern_environmental_impacts,Preference_tolls_or_traffic_limitation,grouped_Frequent_trip_duration_in_minutes,grouped_Region_3,InternetUsers,grouped_Number_vehicles_in_household,grouped_Age
0,Belgium,Female,Upper secondary (high school or similar);,housewife,Not Employed,two,lower middle,Metropolitan area of a big city with more than...,in the suburbs,Well served by public transport,Yes,Maybe yes maybe not,Yes,"Maybe yes, maybe not. I would need to test the...",No,No,Yes,No,No,No,No,No,No,…It is outside an urban area,Make this trip every day/ every working day of...,3-5 KM,5,No preferences,20:21,BE2,75,1,47:51
1,France,Male,"Tertiary and higher (University degree, PhD or...",unemployed,Not Employed,four,low,Small or medium town (less than 250.000 inhabi...,in the centre of the city,Difficult to reach with public transport,Yes,Probably not,Yes,"Maybe yes, maybe not. I would need to test the...",No,No,Yes,No,No,No,No,No,No,"…In an urban area, different from where I live",Make this trip 2-3 days per week,11-20 KM,8,Probably more acceptable to limit road traffic,20:21,FRG,71,3,23:27
2,Czech Republic,Male,"Tertiary and higher (University degree, PhD or...",other employed worker,Employed,four,middle,Large city (from 250.000 to 1.000.000 inhabita...,in the suburbs,Well served by public transport,Yes,Certainly not,Yes,"No, I would not be interested in this service",Yes,Yes,Yes,No,No,No,No,No,No,"…In an urban area, same as where I live",Make this trip every day/ every working day of...,less than 3 KM,6,No preferences,20:21,CZ0,63,1,47:51
3,Sweden,Male,"Tertiary and higher (University degree, PhD or...",teacher/lecturer,Employed,four,higher middle,Metropolitan area of a big city with more than...,in the suburbs,Relatively served by public transport,Yes,Maybe yes maybe not,Unsure/ no answer,Don't know / No answer,No,No,No,No,No,Yes,No,No,No,"…In an urban area, same as where I live",Make this trip every day/ every working day of...,11-20 KM,8,Probably more acceptable to pay for less conge...,58:73,SE1,85,1,54:85
4,Poland,Male,Upper secondary (high school or similar);,manual worker/agricultural worker/farmer,Employed,four,higher middle,Metropolitan area of a big city with more than...,in the suburbs,Well served by public transport,No,Probably yes,No,"Maybe yes, maybe not. I would need to test the...",No,No,No,No,No,No,No,No,Yes,"…In an urban area, same as where I live",Make this trip every day/ every working day of...,11-20 KM,8,No preferences,30:31,PL6,50,1,35:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26600,Cyprus,Female,"Tertiary and higher (University degree, PhD or...",middle manager,Employed,three,lower middle,Rural area,Rural area,Relatively served by public transport,Yes,Don't know/no answer,No,"Yes, instead of purchasing a new car",No,No,Yes,No,No,No,No,No,No,"…In an urban area, different from where I live",Make this trip every day/ every working day of...,31-50 KM,3,Probably more acceptable to pay for less conge...,40:58,CY,63,3,27:30
26601,Cyprus,Female,"Tertiary and higher (University degree, PhD or...",middle manager,Employed,four,middle,Small or medium town (less than 250.000 inhabi...,in the suburbs,Not served by public transport,Yes,Don't know/no answer,No,Don't know / No answer,No,No,Yes,No,No,No,

# Clustering


## DNA selection

In [ ]:
columns = [
  "InternetUsers",
  "Concern_environmental_impacts",
  "grouped_Age",
  "Would_subsribe_car_sharing_if_available", 
  "Preference_tolls_or_traffic_limitation",
  "Gender",
  #"grouped_Region_3", 
  #"Country", 
]
df_DNA = df[columns]
categorical_ix = [3,4,5]

In [ ]:
cleanup_nums = {"Concern_environmental_impacts":
                  {
                      '1': 0,
                      '2': 1,
                      '3': 2,
                      '4': 3,
                      '5': 4,
                      "Don't know": 5,
                      '6': 6,
                      '7': 7,
                      '8': 8,
                      '9': 9,
                      '10': 10 
                  },
                "grouped_Age":
                  {
                      '16:23': 19.5,
                      '23:27': 25,
                      '27:30': 28.5,
                      '30:35': 32.5,
                      '35:39': 37,
                      '39:43': 41,
                      '43:47': 45,
                      '47:51': 49,
                      '51:54': 52.5,
                      '54:85': 69.5
                  },
                "Would_subsribe_car_sharing_if_available":
                  {
                      "Don't know / No answer": 0,
                      'No, I would not be interested in this service': 1,
                      'Maybe yes, maybe not. I would need to test the service before taking a decision': 2,
                      'Yes without any influence on my car ownership': 3,
                      'Yes, instead of purchasing a new car': 4,
                      'Yes and I would give up one car I currently own': 5,
                      "Yes I'm already client of a car sharing service": 6
                  },
                "Preference_tolls_or_traffic_limitation":
                  {
                      'No preferences': 0,
                      'Probably more acceptable to limit road traffic': 1,
                      'Probably more acceptable to pay for less congestion': 2,
                      'Definitely more acceptable to pay for less congestion': 3,
                      'Definitely more acceptable to limit road traffic': 4
                  },
                "Gender":
                  {
                      'Female': 0,
                      'Male': 1
                  }
                }

In [ ]:
df_DNA = df_DNA.replace(cleanup_nums)

In [ ]:
df_DNA.dtypes

InternetUsers                                int64
Concern_environmental_impacts                int64
grouped_Age                                float64
Would_subsribe_car_sharing_if_available      int64
Preference_tolls_or_traffic_limitation       int64
Gender                                       int64
dtype: object

In [ ]:
df_DNA

,InternetUsers,Concern_environmental_impacts,grouped_Age,Would_subsribe_car_sharing_if_available,Preference_tolls_or_traffic_limitation,Gender
0,75,4,49.0,2,0,0
1,71,8,25.0,2,1,1
2,63,6,49.0,1,0,1
3,85,8,69.5,0,2,1
4,50,8,37.0,2,0,1
...,...,...,...,...,...,...
26600,63,2,28.5,4,2,0
26601,63,4,32.5,0,0,0
26602,63,7,32.5,2,0,1
26603,63,7,37.0,4,1,1


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
# Label encoding for country and regions if included
#df_DNA.iloc[:,6] = le.fit_transform(df_DNA.iloc[:,6])
#df_DNA.iloc[:,7] = le.fit_transform(df_DNA.iloc[:,7])
df_DNA

,InternetUsers,Concern_environmental_impacts,grouped_Age,Would_subsribe_car_sharing_if_available,Preference_tolls_or_traffic_limitation,Gender
0,75,4,49.0,2,0,0
1,71,8,25.0,2,1,1
2,63,6,49.0,1,0,1
3,85,8,69.5,0,2,1
4,50,8,37.0,2,0,1
...,...,...,...,...,...,...
26600,63,2,28.5,4,2,0
26601,63,4,32.5,0,0,0
26602,63,7,32.5,2,0,1
26603,63,7,37.0,4,1,1


In [ ]:
x = df_DNA.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_DNA = pd.DataFrame(x_scaled)
df_DNA.columns = columns

In [ ]:
x_original = min_max_scaler.inverse_transform(x_scaled)
df_DNA_original = pd.DataFrame(x_original)

In [ ]:
df_DNA

,InternetUsers,Concern_environmental_impacts,grouped_Age,Would_subsribe_car_sharing_if_available,Preference_tolls_or_traffic_limitation,Gender
0,0.758621,0.4,0.59,0.333333,0.00,0.0
1,0.689655,0.8,0.11,0.333333,0.25,1.0
2,0.551724,0.6,0.59,0.166667,0.00,1.0
3,0.931034,0.8,1.00,0.000000,0.50,1.0
4,0.327586,0.8,0.35,0.333333,0.00,1.0
...,...,...,...,...,...,...
26600,0.551724,0.2,0.18,0.666667,0.50,0.0
26601,0.551724,0.4,0.26,0.000000,0.00,0.0
26602,0.551724,0.7,0.26,0.333333,0.00,1.0
26603,0.551724,0.7,0.35,0.666667,0.25,1.0


In [ ]:
ds_save_path = '/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Survey+dataset/4_DNA_values_v2.csv'

df_DNA.to_csv(ds_save_path, index=False)

# Run until here

## Distance metrics
The implementation of the distance metrics are taken from the library **_distython_**: [https://github.com/KacperKubara/distython](https://github.com/KacperKubara/distython)


In [ ]:
!pip install distython

### Euclidean-Overlap Metric (HEOM)

HEOM is a distance metric taken from [this paper](https://arxiv.org/pdf/cs/9701101.pdf).

HEOM handles categorical, numerical and missing data

$$
d_a(x,y) = 
\begin{equation}
  \begin{cases}
    1, \text{if $x$ or $y$ unknown, else}\\
    overlap(x,y), \text{if $a$ is nominal, else}\\
    normalized\_euclidean\_dist_a(x,y)
  \end{cases}
\end{equation}
$$

$$
overlap(x,y) = 
\begin{equation}
  \begin{cases}
    1, \text{ if x=y}\\
    0, \text{ else}
  \end{cases}
\end{equation}
$$

$$
normalized\_euclidean\_dist_a(x,y) = \frac{|x-y|}{range_a}\\
range_a = max_a - min_a
$$

$$HEOM(x,y) = \sqrt{\sum_{a=1}^m d_a(x_a, y_a)^2}$$
where:
- $x, y$ are datapoints
- $a$ is a feature


In [ ]:
import numpy
import time
class HEOM():
    def __init__(self, X, cat_ix, normalised="normal"):
        """ Heterogeneous Euclidean-Overlap Metric
        Distance metric class which initializes the parameters
        used in heom function
        
        Parameters
        ----------
        X : array-like of shape = [n_rows, n_features]
            Dataset that will be used with HEOM. Needs to be provided
            here because minimum and maximimum values from numerical
            columns have to be extracted
        
        cat_ix : array-like of shape = [cat_columns_number]
            List containing categorical feature indices
        
        cat_ix : array-like of shape = [x]
            List containing missing values indicators
        normalised: string
            normalises euclidan distance function for numerical variables
            Can be set as "std". Default is a column range
        Returns
        -------
        None
        """      
        self.categorical_time = 0
        self.numerical_time = 0
        self.missing_time = 0  
        self.cat_ix = cat_ix
        self.col_ix = [i for i in range(X.shape[1])]
        self.num_ix = np.setdiff1d(self.col_ix, self.cat_ix)
        # Get the normalization scheme for numerical variables
        if normalised == "std":
            self.range = 4* np.nanstd(X, axis = 0)
        else:
            self.range = np.nanmax(X, axis = 0) - np.nanmin(X, axis = 0)
    
    def heom(self, x, y):
        """ Distance metric function which calculates the distance
        between two instances. Handles heterogeneous data and missing values.
        It can be used as a custom defined function for distance metrics
        in Scikit-Learn
        
        Parameters
        ----------
        x : array-like of shape = [n_features]
            First instance 
            
        y : array-like of shape = [n_features]
            Second instance
        Returns
        -------
        result: float
            Returns the result of the distance metrics function
        """

        # Initialise results' array
        results_array = np.zeros(x.shape)
        
        start = time.time() ###
        # Calculate the distance for categorical elements
        results_array[self.cat_ix] = np.not_equal(x[self.cat_ix], y[self.cat_ix]) * 1 # use "* 1" to convert it into int
        end = time.time() ###
        self.categorical_time += (end-start)
        # print(self.cat_ix, results_array[self.cat_ix])

        
        start = time.time() ###
        # Calculate the distance for numerical elements
        results_array[self.num_ix] = np.abs(x[self.num_ix] - y[self.num_ix]) / self.range[self.num_ix]
        end = time.time() ###
        self.numerical_time += (end-start)
        # print(self.num_ix, results_array[self.num_ix])

        # Return the final result
        # Square root is not computed in practice
        # As it doesn't change similarity between instances
        return np.sum(np.square(results_array))

In [ ]:
%load_ext Cython

In [ ]:
from DistanceMetrics.heom_c import HEOM_C

In [ ]:
heom_metric = HEOM(df_DNA[:3], categorical_ix)

heom_metric.heom(df_DNA.iloc[0], df_DNA.iloc[1])

4.111111111111112

In [ ]:
from scipy.spatial.distance import pdist

In [ ]:
pdist(df_DNA[:3], metric=heom_metric.heom)

array([4.11111, 3.11111, 3.88889])

In [ ]:
heom_metric_C = HEOM_C(df_DNA, np.array(categorical_ix))

heom_metric_C.heom(df_DNA.iloc[0], df_DNA.iloc[1])

3.094756242568371

In [ ]:
from DistanceMetrics.heom_c1 import HEOM_C as HEOM_C1
heom_metric_C1 = HEOM_C1(df_DNA, np.array(categorical_ix))

heom_metric_C1.heom(df_DNA.iloc[0].values, df_DNA.iloc[3].values)

4.119726516052319

In [ ]:
from sklearn.metrics import pairwise_distances
dM = pairwise_distances(df_DNA, metric=heom_metric_C1.heom, n_jobs=2)

In [ ]:
#del dM

In [ ]:
import timeit
from sklearn.metrics.pairwise import paired_distances

def heom_c():
  heom_metric_C.heom(x_values, y_values)
def heom_p():
  heom_metric_C1.heom(x_values, y_values)
def euclidean_f():
  #euclidean(x_values, y_values)
  asd = ""
def euclidean_sk():
  paired_distances(x, y)

num_runs = df_DNA.shape[0]

x_values = df_DNA.iloc[0].values
y_values = df_DNA.iloc[1].values
x = df_DNA.iloc[0].to_numpy().reshape(1,-1)
y = df_DNA.iloc[1].to_numpy().reshape(1,-1)
duration_heom_c = timeit.Timer(heom_c).timeit(number = num_runs)
duration_heom_p = timeit.Timer(heom_p).timeit(number = num_runs)
duration_euclidean = timeit.Timer(euclidean_f).timeit(number = num_runs)
duration_euclidean = timeit.Timer(euclidean_sk).timeit(number = num_runs)

avg_duration_heom_c = duration_heom_c/num_runs
avg_duration_heom_p = duration_heom_p/num_runs
avg_duration_euclidean_f = duration_euclidean/num_runs
avg_duration_euclidean_sk = duration_euclidean/num_runs
print(f'HEOM_c on average it took {avg_duration_heom_c} seconds')
print(f'HEOM_p on average it took {avg_duration_heom_p} seconds')
print(f'Euclidean_f on average it took {avg_duration_euclidean_f} seconds')
print(f'Euclidean_sk on average it took {avg_duration_euclidean_sk} seconds')


HEOM_c on average it took 1.0115045968765994e-05 seconds
HEOM_p on average it took 9.168626536361227e-06 seconds
Euclidean_f on average it took 8.277545844773258e-05 seconds
Euclidean_sk on average it took 8.277545844773258e-05 seconds


### Heterogeneous Value Difference Metric (HVDM)

## Hierarchical Clustering

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, inconsistent
from scipy.spatial.distance import cdist, pdist

In [ ]:
def PlotDendrogram(Z, title=""):
    plt.figure(figsize=(25, 10))
    font = {'family' : 'sans', 'size'   : 18}
    plt.rc('font', **font)
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('Sample Index')
    plt.ylabel('Merge Distance')
    dendrogram(Z, 
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=8.,  # font size for the x axis labels
    )
    plt.xticks([])
    plt.show()

In [ ]:
def ComputeWSSBSS(X,Z,k_values=range(1,20)):
    wss_values = []
    bss_values = []

    for k in k_values:
        clustering = fcluster(Z, k, criterion='maxclust')
        frequency = np.bincount(clustering)
        index = np.nonzero(frequency)[0]
        
        centroids = [np.mean(X[clustering==c],axis=0) for c in index]
        
        cdist(X, centroids, 'euclidean')
        D = cdist(X, centroids, 'euclidean')
        cIdx = np.argmin(D,axis=1)
        d = np.min(D,axis=1)

        avgWithinSS = sum(d)/len(X)

        # Total with-in sum of square
        wss = sum(d**2)

        tss = sum(pdist(X)**2)/len(X)

        bss = tss-wss

        wss_values += [wss]
        bss_values += [bss]
    return wss_values,bss_values

In [ ]:
def PlotKneeElbow(bss_values,wss_values,k_values,title=""):
    fig = plt.figure(figsize=(24,12))
    font = {'family' : 'sans', 'size'   : 12}
    plt.rc('font', **font)
    plt.plot(k_values, wss_values, 'bo-', color='red', label='WSS')
    plt.plot(k_values, bss_values, 'bo-', color='blue', label='BSS')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('BSS & WSS')
    plt.xticks(k_values)
    plt.legend()
    plt.title(title);

In [ ]:
Z = {}
#size = -1
Z['Euclidean'] = linkage(df_DNA, 'complete')
Z['HEOM'] = linkage(df_DNA, 'complete', metric=heom_metric_C.heom)

In [ ]:
  for key in Z.keys():
    PlotDendrogram(Z[key],key+' Distance')

In [ ]:
HC_path = '/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Code/Data/HC_complete_heom.csv'

In [ ]:
np.savetxt(HC_path, Z["HEOM"], delimiter=',')

In [ ]:
Z = {}
Z['HEOM'] = np.loadtxt(HC_path, delimiter=',')

100 - 1s

200 - 5s

300 - 10s

400 - 17s

500 - 27s

600 - 38s

700 - 51s

800 - 67s

1000 - 103s - 13s

1200 - 150s

| size | heom | heom_C |
| ---- | ---- | ------ |
| 1000 | 13s | 11s |
| 2000 |  | 43s |
| 5000 |310s| 252s |
|25000 | | |

In [ ]:
wss = {}
bss = {}

for key in Z.keys():
    wss[key],bss[key] = ComputeWSSBSS(df_DNA,Z[key],range(1,20))

In [ ]:
for key in Z.keys():
    PlotKneeElbow(bss[key],wss[key],range(1,20),key+' Distance')

## Plot grids

In [ ]:
cluster = fcluster(Z['HEOM'], 5, criterion='maxclust')
df_DNA_clusters = df_DNA
df_DNA_clusters["cluster"] = cluster
plt.rcParams['figure.figsize'] = (20.0, 10.0)
sns.stripplot(data=df_DNA_clusters, 
            x="Preference_tolls_or_traffic_limitation", 
            y="InternetUsers", 
            hue="cluster",
            jitter=0.4,
            dodge=True,
            size=7,
            palette="icefire",            
            )

In [ ]:
import seaborn as sns
sns.set_theme()

def plotGrid(df_DNA, k, metric, size=None):
  cluster = fcluster(Z[metric], k , criterion='maxclust')
  if size is None:
    df_DNA_clusters = df_DNA
  else:
    df_DNA_clusters = df_DNA[:size]
  df_DNA_clusters["cluster"] = cluster
  
  g = sns.PairGrid(df_DNA_clusters, 
                   hue="cluster",
                   palette="icefire",
                   height=9,
                   aspect=12/6,
                   #x_vars=["Preference_tolls_or_traffic_limitation"],
                   y_vars=["InternetUsers"],
                   )
  g.map_diag(sns.histplot)
  #g.map_offdiag(sns.swarmplot)
  g.map_offdiag(sns.stripplot, jitter=0.4, dodge=True)
  g.add_legend()
  return g

In [ ]:
# 5,7,8,9,11,16 20

In [ ]:
g = plotGrid(df_DNA, 3, 'HEOM')

In [ ]:
g.savefig("/content/drive/MyDrive/MIDA2/IntelligentMobilityProject/Pictures/HC/HC_complete_HEOM.png", dpi=300)